## MNIST Autoencoder mit PyTorch erstellen


##  Import der benötigten Bibliotheken.

In [1]:
import torch
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torch.nn as nn

import matplotlib.pyplot as plt

## Weights and Biases
Zur Nachverfolgung unserer Experimente und zur Visualisierung des Trainingsverlaufs nutzen wir WandB. Dafür wird ein kostenloser Account benötigt. 

Installiere WandB mit( pip install wandb ) und initialisiere es wie folgt:

In [13]:
import wandb
wandb.login()  # nur einmal pro session 
config = {
    "epochs": 40,
    "batch_size": 64,
    "learning_rate": 1e-3,
    "architecture": "Autoencoder",
    "dataset": "MNIST",
    "latent_dim": 10
    }

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: Paste an API key from your profile and hit enter:wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Teilnehmer\_netrc
wandb: Currently logged in as: adel

## Lade den MNIST-Datensatz
Wir verwenden die MNIST-Klasse von torchvision und wandeln die Bilder in Tensoren um

In [15]:
# Load MNIST dataset
transform = ToTensor() # Convert images to PyTorch tensors and normalize to [0, 1]

train_data = MNIST(root='./data', train=True, transform=transform, download=True)
val_data = MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_data, batch_size=config["batch_size"], shuffle=True)
val_loader = DataLoader(val_data, batch_size=config["batch_size"], shuffle=True)

100%|██████████| 9.91M/9.91M [00:04<00:00, 2.09MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 309kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 1.65MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 1.44MB/s]


## Definiere das Autoencoder-Modell
Der Autoencoder besteht aus zwei Hauptkomponenten:

Encoder: Komprimiert das 28x28-Bild zu einem latenten Vektor.

Decoder: Rekonstruiert das Bild aus diesem Vektor.

Wir verwenden lineare Schichten mit ReLU-Aktivierungen und schließen den Decoder mit einer Sigmoid-Aktivierung ab.

In [16]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, config["latent_dim"])
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(config["latent_dim"], 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 28 * 28),
            nn.Sigmoid()
        )

    def forward(self, x):
        x_flat = x.flatten(start_dim=1)       # 2D-Bild → 1D-Vektor (784)
        encoded = self.encoder(x_flat)        # Komprimieren
        decoded = self.decoder(encoded)       # Wiederherstellen
        return decoded.reshape_as(x)          # Zurück in 28x28 Form bringen
